In [266]:
import pandas as pd
import numpy as np
import time
import datetime

dimension_path = "../DimensionsPre/"
data_path = "../Data/"

In [267]:
Date_dimension = pd.read_csv(dimension_path+"Date_dimension/Date_dimension.csv")
Mobility_dimension = pd.read_csv(dimension_path+"Mobility_dimension/Mobility_dimension.csv")
Patient_dimension = pd.read_csv(dimension_path+"Patient_dimension/Patient_dimension.csv")
PHU_Location_dimension = pd.read_csv(dimension_path+"PHU_Location_dimension/PHU_Location_dimension.csv")
Special_Measures_dimension = pd.read_csv(dimension_path+"Special_Measures_dimension/Special_Measures_dimension.csv")
Weather_dimension = pd.read_csv(dimension_path+"Weather_dimension/Weather_dimension.csv")

In [268]:
cases = pd.read_csv(data_path+"IndividualCasesInOntario.csv")
# cases.head()

In [269]:
cases["Accurate_Episode_Date"] = pd.to_datetime(cases["Accurate_Episode_Date"]).dt.strftime('%Y-%m-%d')
cases["Case_Reported_Date"] = pd.to_datetime(cases["Case_Reported_Date"]).dt.strftime('%Y-%m-%d')
cases["Test_Reported_Date"] = pd.to_datetime(cases["Test_Reported_Date"]).dt.strftime('%Y-%m-%d')
cases["Specimen_Date"] = pd.to_datetime(cases["Specimen_Date"]).dt.strftime('%Y-%m-%d')
cases = cases[(cases["Test_Reported_Date"] < "2021-03-01") & (cases["Test_Reported_Date"] >= "2020-11-01")]
# cases = cases[(cases["Reporting_PHU_City"] == "Toronto") | (cases["Reporting_PHU_City"] == "Ottawa")]
durhanCases = cases[(cases["Reporting_PHU"].map(lambda x: "durham" in x.lower()))   |  (cases["Reporting_PHU_City"].map(lambda x: "durham" in x.lower()))]
haltonCases = cases[(cases["Reporting_PHU"].map(lambda x: "halton" in x.lower()))   |  (cases["Reporting_PHU_City"].map(lambda x: "halton" in x.lower()))]
peelCases = cases[(cases["Reporting_PHU"].map(lambda x: "peel" in x.lower()))   |  (cases["Reporting_PHU_City"].map(lambda x: "peel" in x.lower()))]
yorkCases = cases[(cases["Reporting_PHU"].map(lambda x: "york" in x.lower()))   |  (cases["Reporting_PHU_City"].map(lambda x: "york" in x.lower()))]
ottawaCases = cases[(cases["Reporting_PHU"].map(lambda x: "ottawa" in x.lower()))   |  (cases["Reporting_PHU_City"].map(lambda x: "ottawa" in x.lower()))]
torontoCases = cases[(cases["Reporting_PHU"].map(lambda x: "toronto" in x.lower()))   |  (cases["Reporting_PHU_City"].map(lambda x: "toronto" in x.lower()))]
frames = [durhanCases, haltonCases, peelCases, yorkCases, ottawaCases, torontoCases]
cases = pd.concat(frames)
cases = cases.fillna({'Outbreak_Related':'Unknown'})#[:67401:100]

In [270]:
cases[["Reporting_PHU","Reporting_PHU_City"]].drop_duplicates()

,Reporting_PHU,Reporting_PHU_City
2940,Durham Region Health Department,Whitby
2967,Halton Region Health Department,Oakville
2918,Peel Public Health,Mississauga
2920,York Region Public Health Services,Newmarket
5005,Ottawa Public Health,Ottawa
2950,Toronto Public Health,Toronto


In [271]:
reportingPHU2SubRegion2={
    "Durham Region Health Department":"Regional Municipality of Durham",
    "Halton Region Health Department":"Regional Municipality of Halton",
    "Peel Public Health":"Regional Municipality of Peel",
    "York Region Public Health Services":"Regional Municipality of York",
    "Ottawa Public Health":"Ottawa Division",
    "Toronto Public Health":"Toronto Division"
}
reportingPHUCity2StationName={
    "Durham Region Health Department":"TORONTO NORTH YORK",
    "Halton Region Health Department":"TORONTO NORTH YORK",
    "Peel Public Health":"TORONTO NORTH YORK",
    "York Region Public Health Services":"TORONTO NORTH YORK",
    "Ottawa Public Health":"OTTAWA INTL A",
    "Toronto Public Health":"TORONTO NORTH YORK"
}
fact_table =  pd.DataFrame(columns=['Onset_date_key','Reported_date_key','Test_date_key', 'Specimen_date_key', 'Patient_key', "PHU_location_key","Special_measure_key","Mobility_key","Weather_key","Resolved","Unresolved","Fatal"])

In [272]:
#TO REMOVED:df.dropna(axis=0,how='any')
start = time.time()
cases = cases.dropna(axis=0,how='any')
# cases = cases.head()
rows = 0
Resolved,Unresolved,Fatal = 0,0,0
specialMeasureMismatch, mobilityMismatch, weatherMismatch = 0,0,0
tmp = 0 

for idx, row in cases.iterrows():
    rows += 1
    
    #not considering 2021 data
#     if (row["Case_Reported_Date"][:4]!="2020"):
#         continue
    
    if (rows%100 == 0):
        percentage = str(round((rows/len(cases))*100, 2)  )
        remaining = (time.time()-start)*((len(cases)-rows)/rows)
        print("%s/%s - %s%%  --about %s:%s left"%(str(rows),str(len(cases)),percentage,str(int(remaining//60)),str(int(remaining%60))),end = "\r")
    
    if (row["Outcome1"] == "Resolved"):
        Resolved+=1
    elif (row["Outcome1"] == "Not Resolved"):
        Unresolved+=1
    else:
        Fatal+=1

        
    # Special Measure
    try:
        specialMeasureId = Special_Measures_dimension[(Special_Measures_dimension["Reporting_PHU_id"]==row["Reporting_PHU_ID"])&(Special_Measures_dimension["Start-date"]<=row["Case_Reported_Date"])&(Special_Measures_dimension["End-date"]>=row["Case_Reported_Date"])]["surrogate_key"].values[0]
    except IndexError as e:        
        specialMeasureMismatch+=1
        specialMeasureId = None
        
    # Mobility
    try:
        Mobility_dimensionId = Mobility_dimension[(Mobility_dimension["sub_region_2"] == reportingPHU2SubRegion2[row["Reporting_PHU"]])&(Mobility_dimension["date"]==row["Case_Reported_Date"])]["surrogate_key"].values[0]
    except IndexError as e:        
        mobilityMismatch+=1
        Mobility_dimensionId = None
        
       
    # Weather
    try:
        Weather_dimensionId = Weather_dimension[(Weather_dimension["Station Name"]==reportingPHUCity2StationName[row["Reporting_PHU"]])&(Weather_dimension["Date/Time"]==row["Case_Reported_Date"])]["surrogate_key"].values[0]
    except IndexError as e:        
        weatherMismatch+=1
        Weather_dimensionId = None

        

    fact_row = [
    Date_dimension[Date_dimension["date"]==row["Accurate_Episode_Date"]]["surrogate_key"].values[0],
    Date_dimension[Date_dimension["date"]==row["Case_Reported_Date"]]["surrogate_key"].values[0],
    Date_dimension[Date_dimension["date"]==row["Test_Reported_Date"]]["surrogate_key"].values[0],
    Date_dimension[Date_dimension["date"]==row["Specimen_Date"]]["surrogate_key"].values[0],
    Patient_dimension[(Patient_dimension["Age_Group"]==row["Age_Group"])&(Patient_dimension["Client_Gender"]==row["Client_Gender"])&(Patient_dimension["Case_AcquisitionInfo"]==row["Case_AcquisitionInfo"])&(Patient_dimension["Outbreak_Related"]==row["Outbreak_Related"])]["surrogate_key"].values[0],
    PHU_Location_dimension[PHU_Location_dimension["Reporting_PHU_ID"]==row["Reporting_PHU_ID"]]["surrogate_key"].values[0],

    specialMeasureId,
    Mobility_dimensionId,
    Weather_dimensionId,
        
    Resolved,Unresolved,Fatal
    ]
    fact_table.loc[len(fact_table)] = fact_row

   
print("Finished Forming Fact Table Took %s:%s to Finish"%(int((time.time()-start)//60),int((time.time()-start)%60)))
print("Special Measure Mismatch: %s"%specialMeasureMismatch)
print("Mobility Mismatch: %s"%mobilityMismatch)
print("Weather Mismatch: %s"%weatherMismatch)
# print("Tmp : %s"%tmp )
print("Total Cases: %s"%len(cases))
        
        
        
        
        

Finished Forming Fact Table Took 63:48 to Finish
Special Measure Mismatch: 3613
Mobility Mismatch: 3
Weather Mismatch: 68
Total Cases: 151566


In [273]:
fact_table.shape

(151566, 12)

In [274]:
fact_table.isna().sum()

Onset_date_key            0
Reported_date_key         0
Test_date_key             0
Specimen_date_key         0
Patient_key               0
PHU_location_key          0
Special_measure_key    3613
Mobility_key              3
Weather_key              68
Resolved                  0
Unresolved                0
Fatal                     0
dtype: int64

In [275]:
fact_table.dropna(axis=0,how='any').shape

(147904, 12)

In [276]:
fact_table = fact_table.dropna(axis=0,how='any')

In [277]:
fact_table.isna().sum()

Onset_date_key         0
Reported_date_key      0
Test_date_key          0
Specimen_date_key      0
Patient_key            0
PHU_location_key       0
Special_measure_key    0
Mobility_key           0
Weather_key            0
Resolved               0
Unresolved             0
Fatal                  0
dtype: int64

In [278]:
fact_table.shape

(147904, 12)

In [279]:
fact_table["Special_measure_key"].drop_duplicates()

0         15
57        16
76        17
187       13
194       14
8728      41
8796      43
8889      40
8977      42
15837     97
16313     96
16449     98
19262     99
62510    165
62732    167
62734    168
62816    166
62989    164
84217     93
84272     94
84353     95
91717    149
91740    150
92085    147
92092    148
Name: Special_measure_key, dtype: object

In [280]:
fact_table["Special_measure_key"].isna().sum()

0

In [281]:
Weather_dimension["Station Name"].drop_duplicates()

0          OTTAWA INTL A
61    TORONTO NORTH YORK
Name: Station Name, dtype: object

In [282]:
fact_table["PHU_location_key"].drop_duplicates()

0        0
8728     1
15837    2
62510    3
84217    4
91717    5
Name: PHU_location_key, dtype: object

In [283]:
(time.time()-start)/60

63.81770279804866